In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
dataDir = microsimDir+"/NOTEBOOKS/DATA"

#contains person-year information from a Microsim NHANES simulation
df = pd.read_csv(dataDir+"/nhanes-normality-test-4.csv")

In [2]:
#df.columns.to_list()

In [3]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [4]:
#for col in categoricalVars:
#    print(df[col].value_counts())

In [5]:
%%time
nCategories = 0
sizesList = list()
for gender in set(df["gender"].tolist()):
    for smoking in set(df["smokingStatus"].tolist()):
        print(gender,smoking)
        for raceEthnicity in set(df["raceEthnicity"].tolist()):
            for statin in set(df["statin"].tolist()):
                for education in set(df["education"].tolist()):
                    for alcoholPerWeek in set(df["alcoholPerWeek"].tolist()):
                        for anyPhysicalActivity in set(df["anyPhysicalActivity"].tolist()):
                            for antiHypertensiveCount in set(df["antiHypertensiveCount"].tolist()):
                                
                                nCategories += 1
                                
                                dfForGroup = df.loc[(df["gender"]==gender) & 
                                                        (df["smokingStatus"]==smoking) &
                                                        (df["raceEthnicity"]==raceEthnicity) &
                                                        (df["statin"]==statin) &
                                                        (df["education"]==education) &
                                                        (df["alcoholPerWeek"]==alcoholPerWeek) &
                                                        (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                                        (df["antiHypertensiveCount"]==antiHypertensiveCount), :].copy()
                                
                                size = dfForGroup.shape[0]
                                
                                #data = np.array(df.loc[(df["gender"]==gender) & 
                                #                        (df["smokingStatus"]==smoking) &
                                #                        (df["raceEthnicity"]==raceEthnicity) &
                                #                        (df["statin"]==statin) &
                                #                        (df["education"]==education) &
                                #                        (df["alcoholPerWeek"]==alcoholPerWeek) &
                                #                        (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                #                        (df["antiHypertensiveCount"]==antiHypertensiveCount), 
                                #                       continuousVars].copy())
                                
                                #size = data.shape[0]
                                
                                if size>0:
                                    group = [gender, smoking, raceEthnicity, statin, education, alcoholPerWeek, 
                                         anyPhysicalActivity, antiHypertensiveCount]
                                    names = dfForGroup["name"].tolist()
                                    sizesList += [[group, size, names]]
                                                    

1 0
1 1
1 2
2 0
2 1
2 2
CPU times: user 5.99 s, sys: 4.12 ms, total: 5.99 s
Wall time: 6 s


In [6]:
groupsList = sizesList
totalPeople = sum(list(map( lambda x: x[1], sizesList)))

In [ ]:
#trying this out...for now....

with (open(dataDir+"/nhanesMeans-4.csv", "w") as meansFile, open(dataDir+"/nhanesCovs-4.csv","w") as covsFile,
      open(dataDir+"/nhanesMin-4.csv", "w") as minFile, open(dataDir+"/nhanesMax-4.csv", "w") as maxFile):
    pd.DataFrame(categoricalVars+["weight"]+continuousVars).T.to_csv(meansFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(covsFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(minFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(maxFile, header=False, index=False)
    for i,groupWithSize in enumerate([groupsList[0]]):
        if i%500==0:
            print(i)
        group = groupWithSize[0]
        size = groupWithSize[1]
    
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
    
        dataDf = df.loc[(df["gender"]==gender) & 
                                (df["smokingStatus"]==smoking) &
                                (df["raceEthnicity"]==raceEthnicity) &
                                (df["statin"]==statin) &
                                (df["education"]==education) &
                                (df["alcoholPerWeek"]==alcoholPerWeek) &
                                (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                (df["antiHypertensiveCount"]==antiHypertensiveCount), 
                                                        continuousVars+["name"]].copy()
        data = np.array(dataDf)
        
        

In [ ]:
dataDf.head()

In [ ]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})
dataDf = dataDf.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()

In [ ]:
dataDf["weight"] = dataDf["WTINT2YR"].astype('int')
dataDf.head()

In [ ]:
for row in range(dataDf.shape[0]):
    
    copied_df = pd.DataFrame(dataDf.iloc[row].repeat(2)).reset_index(drop=True)

    # Concatenate the original DataFrame with the copied rows
    dataDf = pd.concat([dataDf, copied_df], ignore_index=True)

In [ ]:
copied_df.T

In [ ]:
dataDf.head()

In [ ]:
dataDf.applymap(lambda x: np.random.default_rng().uniform(0.8*x, 1.2*x))

In [7]:
with (open(dataDir+"/nhanesMeans-4.csv", "w") as meansFile, open(dataDir+"/nhanesCovs-4.csv","w") as covsFile,
      open(dataDir+"/nhanesMin-4.csv", "w") as minFile, open(dataDir+"/nhanesMax-4.csv", "w") as maxFile):
    pd.DataFrame(categoricalVars+["weight"]+continuousVars).T.to_csv(meansFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(covsFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(minFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(maxFile, header=False, index=False)
    for i,groupWithSize in enumerate(groupsList):
        if i%500==0:
            print(i)
        group = groupWithSize[0]
        size = groupWithSize[1]
    
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
    
        data = np.array(df.loc[(df["gender"]==gender) & 
                                (df["smokingStatus"]==smoking) &
                                (df["raceEthnicity"]==raceEthnicity) &
                                (df["statin"]==statin) &
                                (df["education"]==education) &
                                (df["alcoholPerWeek"]==alcoholPerWeek) &
                                (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                (df["antiHypertensiveCount"]==antiHypertensiveCount), 
                                                        continuousVars].copy())
    
        distMean, distCov = multivariate_normal.fit(data)
        
        distMinDf = pd.DataFrame( np.min(data,axis=0) )
        pd.concat(
                [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T,
                      distMinDf.T],axis=1, ignore_index=True).to_csv(minFile, header=False, index=False)
        minFile.write("\n\n")
        
        distMaxDf = pd.DataFrame( np.max(data,axis=0) )
        pd.concat(
                [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T,
                      distMaxDf.T],axis=1, ignore_index=True).to_csv(maxFile, header=False, index=False)
        maxFile.write("\n\n")
        
        #1 row per block because means are of shape (1,4)
        distMeanDf = pd.DataFrame(distMean)
        pd.concat(
                    [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                    education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount, size/totalPeople]).T,
                      distMeanDf.T],axis=1, ignore_index=True).to_csv(meansFile, header=False, index=False)
        meansFile.write("\n\n")
                    
        #4 rows per block because covs are of shape (4,4)
        distCovDf = pd.DataFrame(distCov)
        pd.concat(
                [ pd.concat([pd.DataFrame([gender, smoking, raceEthnicity, statin,
                                           education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T]*len(continuousVars),ignore_index=True),
                  distCovDf.T],axis=1, ignore_index=True).to_csv(covsFile, header=False, index=False)
        covsFile.write("\n\n")

0
500
1000
1500


In [8]:
meansDf = pd.read_csv(dataDir+"/nhanesMeans-4.csv", header=0)
covsDf = pd.read_csv(dataDir+"/nhanesCovs-4.csv", header=0)
minDf = pd.read_csv(dataDir+"/nhanesMin-4.csv", header=0)
maxDf = pd.read_csv(dataDir+"/nhanesMax-4.csv", header=0)

In [9]:
#covsDf.head()
minDf.head()
#meansDf.head()

,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,1,0,1,False,1,0,0,0.0,22.0,35.0,21.76,146.0,67.0,4.8,83.0,78.3,0.6,100.666667,48.666667
1,1,0,1,False,1,0,0,1.0,62.0,35.0,25.58,187.0,69.0,4.8,96.0,95.0,0.6,139.333333,73.333333
2,1,0,1,False,1,0,0,2.0,71.0,42.0,23.78,150.0,130.0,6.9,59.0,92.3,8.5,177.333333,101.333333
3,1,0,1,False,1,0,0,3.0,59.0,47.0,22.50,243.0,178.0,5.0,160.0,86.1,0.7,126.666667,78.000000
4,1,0,1,False,1,0,1,0.0,24.0,33.0,23.44,190.0,100.0,5.2,111.0,90.4,0.4,98.000000,67.333333


In [10]:
meansDf.head()

,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,weight,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,1,0,1,False,1,0,0,0.0,0.001652,51.111111,42.111111,27.175556,220.888889,139.111111,5.833333,146.777778,97.822222,0.777778,130.666667,71.333333
1,1,0,1,False,1,0,0,1.0,0.000551,68.666667,53.000000,28.263333,191.000000,99.333333,5.200000,111.000000,101.400000,0.766667,159.777778,76.000000
2,1,0,1,False,1,0,0,2.0,0.000184,71.000000,42.000000,23.780000,150.000000,130.000000,6.900000,59.000000,92.300000,8.500000,177.333333,101.333333
3,1,0,1,False,1,0,0,3.0,0.000367,69.000000,56.500000,34.345000,246.000000,201.000000,5.500000,161.500000,113.450000,0.750000,146.333333,86.000000
4,1,0,1,False,1,0,1,0.0,0.001101,55.333333,38.166667,27.845000,213.333333,202.666667,5.650000,133.333333,99.650000,0.760000,129.333333,76.000000


In [11]:
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    
    gender = group[0]
    smoking = group[1]
    raceEthnicity = group[2]
    statin = group[3]
    education = group[4]
    alcoholPerWeek = group[5]
    anyPhysicalActivity = group[6] 
    antiHypertensiveCount = group[7]
    
    gCov = covsDf.loc[ (covsDf["gender"]==gender) &
             (covsDf["smokingStatus"]==smoking) &
             (covsDf["raceEthnicity"]==raceEthnicity) &
             (covsDf["statin"]==statin) &
             (covsDf["education"]==education) &
             (covsDf["alcoholPerWeek"]==alcoholPerWeek) &
             (covsDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (covsDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    #if np.linalg.matrix_rank(gCov) < gCov.shape[0]:
    #    groupWithSize += [True]
    #else:
    #    groupWithSize += [False]
        
    #if (np.linalg.matrix_rank(gCov) < gCov.shape[0]) | (not np.all(np.linalg.eig(gCov)[0]>0)) | (not np.allclose(gCov, gCov.T)):
    if not np.all(np.linalg.eig(gCov)[0]>10**(-3)):
        groupWithSize += [True]
    else:
        groupWithSize += [False]

In [12]:
sum(list(map(lambda x: x[3], groupsList)))

1531

In [13]:
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    singular = groupWithSize[3]
    
    if singular:
        #print(group)
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
        
        gMean = meansDf.loc[ (meansDf["gender"]==gender) &
             (meansDf["smokingStatus"]==smoking) &
             (meansDf["raceEthnicity"]==raceEthnicity) &
             (meansDf["statin"]==statin) &
             (meansDf["education"]==education) &
             (meansDf["alcoholPerWeek"]==alcoholPerWeek) &
             (meansDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (meansDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
        
        altGroupsList = list()
        for altGroupWithSize in groupsList:
            altGroup = altGroupWithSize[0]
            altSize = altGroupWithSize[1]
            altSingular = altGroupWithSize[3]
            
            if not altSingular:
                altGender = altGroup[0]
                altSmoking = altGroup[1]
                altRaceEthnicity = altGroup[2]
                altStatin = altGroup[3]
                altEducation = altGroup[4]
                altAlcoholPerWeek = altGroup[5]
                altAnyPhysicalActivity = altGroup[6] 
                altAntiHypertensiveCount = altGroup[7]
    
                altGMean = meansDf.loc[ (meansDf["gender"]==altGender) &
                                     (meansDf["smokingStatus"]==altSmoking) &
                                     (meansDf["raceEthnicity"]==altRaceEthnicity) &
                                     (meansDf["statin"]==altStatin) &
                                     (meansDf["education"]==altEducation) &
                                     (meansDf["alcoholPerWeek"]==altAlcoholPerWeek) &
                                     (meansDf["anyPhysicalActivity"]==altAnyPhysicalActivity) &
                                     (meansDf["antiHypertensiveCount"]==altAntiHypertensiveCount), continuousVars]
    
                altGCov = covsDf.loc[ (covsDf["gender"]==altGender) &
                                   (covsDf["smokingStatus"]==altSmoking) &
                                   (covsDf["raceEthnicity"]==altRaceEthnicity) &
                                   (covsDf["statin"]==altStatin) &
                                   (covsDf["education"]==altEducation) &
                                   (covsDf["alcoholPerWeek"]==altAlcoholPerWeek) &
                                   (covsDf["anyPhysicalActivity"]==altAnyPhysicalActivity) &
                                   (covsDf["antiHypertensiveCount"]==altAntiHypertensiveCount), continuousVars]
    
                gDist = multivariate_normal(np.array(altGMean)[0], np.array(altGCov), allow_singular=False)
        
                altGroupsList += [[altGroup, gDist.pdf(gMean)]] #[altGroup, gDist.pdf(group)]
    
        probabilities = list(map(lambda x: x[1], altGroupsList))
        maxProb = max(probabilities)
        altGroupWithMaxProb = altGroupsList[probabilities.index(maxProb)][0]
        groupWithSize += [altGroupWithMaxProb]
    else:
        groupWithSize += [None]

In [14]:
groupsList[0], len(groupsList)

([[1, 0, 1, False, 1, 0, 0, 0.0],
  9,
  [41909, 42688, 43025, 43390, 44501, 44548, 45255, 45756, 45914],
  True,
  [1, 2, 1, False, 1, 3, 1, 0.0]],
 1611)

In [19]:
personsDf = pd.DataFrame(data=None, columns= ["name"]+categoricalVars+continuousVars)
#popSize = 100
for iii, groupWithSize in enumerate(groupsList):
    if iii%100==0:
        print(iii)
    group = groupWithSize[0]
    size = groupWithSize[1]
    names = groupWithSize[2]
    singular = groupWithSize[3]
    altGroup = groupWithSize[4]
    
    if singular:
        gender = altGroup[0]
        smoking = altGroup[1]
        raceEthnicity = altGroup[2]
        statin = altGroup[3]
        education = altGroup[4]
        alcoholPerWeek = altGroup[5]
        anyPhysicalActivity = altGroup[6] 
        antiHypertensiveCount = altGroup[7]
    else:
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
    
    gMean = meansDf.loc[ (meansDf["gender"]==gender) &
             (meansDf["smokingStatus"]==smoking) &
             (meansDf["raceEthnicity"]==raceEthnicity) &
             (meansDf["statin"]==statin) &
             (meansDf["education"]==education) &
             (meansDf["alcoholPerWeek"]==alcoholPerWeek) &
             (meansDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (meansDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    gCov = covsDf.loc[ (covsDf["gender"]==gender) &
             (covsDf["smokingStatus"]==smoking) &
             (covsDf["raceEthnicity"]==raceEthnicity) &
             (covsDf["statin"]==statin) &
             (covsDf["education"]==education) &
             (covsDf["alcoholPerWeek"]==alcoholPerWeek) &
             (covsDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (covsDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    gDist = multivariate_normal(np.array(gMean)[0], np.array(gCov), allow_singular=True)
    
    if size>4:
        gMin = minDf.loc[ (minDf["gender"]==group[0]) &
                 (minDf["smokingStatus"]==group[1]) &
                 (minDf["raceEthnicity"]==group[2]) &
                 (minDf["statin"]==group[3]) &
                 (minDf["education"]==group[4]) &
                 (minDf["alcoholPerWeek"]==group[5]) &
                 (minDf["anyPhysicalActivity"]==group[6]) &
                 (minDf["antiHypertensiveCount"]==group[7]), continuousVars]
    
        gMax = maxDf.loc[ (maxDf["gender"]==group[0]) &
                 (maxDf["smokingStatus"]==group[1]) &
                 (maxDf["raceEthnicity"]==group[2]) &
                 (maxDf["statin"]==group[3]) &
                 (maxDf["education"]==group[4]) &
                 (maxDf["alcoholPerWeek"]==group[5]) &
                 (maxDf["anyPhysicalActivity"]==group[6]) &
                 (maxDf["antiHypertensiveCount"]==group[7]), continuousVars]
    else:
        gMin = minDf.loc[ (minDf["gender"]==gender) &
                 (minDf["smokingStatus"]==smoking) &
                 (minDf["raceEthnicity"]==raceEthnicity) &
                 (minDf["statin"]==statin) &
                 (minDf["education"]==education) &
                 (minDf["alcoholPerWeek"]==alcoholPerWeek) &
                 (minDf["anyPhysicalActivity"]==anyPhysicalActivity) &
                 (minDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
        gMax = maxDf.loc[ (maxDf["gender"]==gender) &
                 (maxDf["smokingStatus"]==smoking) &
                 (maxDf["raceEthnicity"]==raceEthnicity) &
                 (maxDf["statin"]==statin) &
                 (maxDf["education"]==education) &
                 (maxDf["alcoholPerWeek"]==alcoholPerWeek) &
                 (maxDf["anyPhysicalActivity"]==anyPhysicalActivity) &
                 (maxDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    #nDraws = int(popSize*size/totalPointsInGroups)
    nDraws = size
    nDrawsNeeded = nDraws
    draws=None
    while nDrawsNeeded!=0:
        if draws is None:
            draws = gDist.rvs(size=nDrawsNeeded)
        else:
            #print(f"{i} redrawing")
            if len(draws.shape)==1:
                draws = draws.reshape((1,len(continuousVars)))
            if (nDrawsNeeded==1):
                draws = np.concatenate( (draws, gDist.rvs(size=nDrawsNeeded).reshape((1,len(continuousVars)))), axis=0 )
            else:
                draws = np.concatenate( (draws, gDist.rvs(size=nDrawsNeeded)), axis=0 )
           
        if nDraws==1:
            draws = draws.reshape((1,len(continuousVars)))
            
        rowsOutOfBounds = np.array([False]*nDraws)
        for i, bound in enumerate(np.array(gMin)[0]):
            rowsOutOfBounds = rowsOutOfBounds | (draws[:,i]<0.7*bound)
        for i, bound in enumerate(np.array(gMax)[0]):
            rowsOutOfBounds = rowsOutOfBounds | (draws[:,i]>1.3*bound)
        nDrawsNeeded = nDraws - np.sum(~rowsOutOfBounds)
        draws = draws[~rowsOutOfBounds,:]    
    
    if nDraws == 0:
        pass
    elif nDraws==1:
        #print(nDraws)
        #personRowCont = pd.DataFrame(draws).T
        personRowCont = pd.DataFrame(draws)
    else:
        #print(nDraws)
        personRowCont = pd.DataFrame(draws)
    if nDraws>0:
        personRowCont.columns=continuousVars
        personRowCat = pd.concat([pd.DataFrame(group).T]*nDraws, ignore_index=True)
        personRowCat.columns = categoricalVars
        personRow = pd.concat( [pd.Series(names), personRowCat, personRowCont], axis=1).rename(columns={0:"name"})
        personsDf = pd.concat([personsDf,personRow])
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


In [20]:
personsDf["age"] = round(personsDf["age"]).astype('int')

In [21]:
personsDf.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,40,59.860116,30.433631,243.771416,60.782933,5.915046,154.068838,98.907992,0.859896,117.463015,80.630951
1,42688,1,0,1,False,1,0,0,0.0,66,49.866495,26.274580,252.707847,93.523817,6.556322,170.343466,94.925142,0.906850,144.930624,78.151846
2,43025,1,0,1,False,1,0,0,0.0,58,51.263620,28.790801,207.808584,168.141061,6.219358,122.685090,100.727180,0.954121,126.312875,68.255347
3,43390,1,0,1,False,1,0,0,0.0,41,53.007622,29.412992,228.923683,65.223746,5.654073,144.108985,100.058501,0.775893,127.072760,76.350982
4,44501,1,0,1,False,1,0,0,0.0,51,53.265911,21.477110,179.991013,119.950624,5.342257,110.602062,80.954864,0.631021,129.142989,69.216155


In [22]:
personsDf.to_csv(dataDir+"/nhanes-persons-from-Gaussians.csv", index=False)